In [0]:
import torch
import torchvision
from tqdm import tqdm



In [33]:
%cd /content/
!mkdir BG_FG_mask


/content


In [0]:
%rm -r /content/BG_FG/

In [0]:
!unzip -q '/content/drive/My Drive/files/bg_fg.zip' -d /content/BG_FG

In [30]:
import glob
import os

inputFolder= '/content/BG_FG/bg'
os.mkdir('Resized_FG_mask')
%cd Resized_FG_mask


/content/Resized_FG_mask


In [0]:
import cv2
os.mkdir('fg_masked')

i=0

for img in glob.glob(inputFolder+"/*.jpg"):
  image = cv2.imread(img)
  imgrez=cv2.resize(image,(224,224))
  cv2.imwrite('fg_masked/img%04i.jpg'%i,imgrez)
  i+=1

In [39]:
dataset = torchvision.datasets.ImageFolder(root='/content/BG_FG',
                                           transform = torchvision.transforms.ToTensor())
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1024,
    num_workers=4,
    shuffle=False,
    pin_memory= True
)

mean = 0.0
for data, _ in tqdm(loader, desc="Computing mean"):
    batch_samples = data.size(0)
    # Flatten each channel
    data = data.view(batch_samples, data.size(1), -1)
    # Since the denominator (224*224) is constant, instead of doing sum first and
    # then taking mean, we can directly take mean and sum it.
    mean += data.mean(2).sum(0)

mean /= len(loader.dataset)
print("\nMean: ", mean)

std = 0.0
for data, _ in tqdm(loader, desc="Computing std"):
    batch_samples = data.size(0)
    # Flatten each channel
    data = data.view(batch_samples, data.size(1), -1)
    # Sum (x-mean)^2 per channel across all batches pixels
    std += ((data - mean.unsqueeze(1))**2).sum([0,2])

std = torch.sqrt(std / (len(loader.dataset)*224*224))
print("\nStd: ", std)


Computing mean: 100%|██████████| 4/4 [00:09<00:00,  2.33s/it]

Computing std:   0%|          | 0/4 [00:00<?, ?it/s]


Mean:  tensor([0.1727, 0.1361, 0.1439])



Computing std: 100%|██████████| 4/4 [00:10<00:00,  2.68s/it]


Std:  tensor([0.1907, 0.1712, 0.1576])
